# Emergency Networking API - Connection and data fetch

In [1]:
# Import modules
import http.client
import pandas as pd
import json

## Make a connection to the Emergency Networking API and fetch data

This is a simple example of how to connect to the Emergency Networking API and fetch data. The API is a RESTful API that provides access to a database of emergency contacts and resources. The API is free to use and does not require an API key.

In [2]:
# Setup main variables

with open('assets/token.txt', 'r') as file:
    token = file.read()
conn = http.client.HTTPSConnection("staging.emergencynetworking.com")

headers = {
    'Accept': "application/json",
    'Authorization': f"Bearer {token}"
}
print("Requesting data from API")

# API call
conn.request("GET", "/department-api/v/incidents", headers=headers)

res = conn.getresponse()
data = res.read()
data_obj = json.loads(data.decode("utf-8"))

# print dataObject attributes 'total', ''perPage', 'page', 'data'
print("Data received")
print(f"Total records: {data_obj['total']}")
print(f"Records per page: {data_obj['perPage']}")
print(f"Page: {data_obj['currentPage']}")

# transform dataObj['data'] to pandas dataframe
df = pd.DataFrame(data_obj['data'])
df.head(5)


Requesting data from API
Data received
Total records: 22053
Records per page: 25
Page: 1


,id,dispatched_as,station,command_established,location_address_unit,360_evaluation_complete,location_address_unit_type,first_water,primary_search,controlled,...,shift,district,primary_action_taken,additional_actions_taken,narrative,officer_in_charge,units,current_status,created_at,updated_at
0,7c598630-ee34-11ee-9199-ffd8fb9b0f4c,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],EMS call with pt transport.,1008,[{'id': '9dbe6ac4-18b8-405a-9561-d8cb7ec95e0f'...,archived,2022-09-16T00:48:42.000000Z,2024-07-30T18:25:28.000000Z
1,7c60a400-ee34-11ee-98ae-f561702ccec3,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,32,[32],EMS call with pt transport.,1008,[{'id': '386be940-dc67-4727-9a84-89401c026fcb'...,archived,2022-09-16T02:41:13.000000Z,2024-07-30T18:25:28.000000Z
2,7c6826c0-ee34-11ee-8510-f52c0e7fd3fa,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],Medical transport.,948,[{'id': 'e73bf29c-0d51-4055-b142-27663114742d'...,archived,2022-09-16T02:48:44.000000Z,2024-07-30T18:25:28.000000Z
3,7c6d52e0-ee34-11ee-be6a-e92e937dd693,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,34,[34],EMS call and transport.,958,[{'id': 'd9e606a7-0e05-42e5-b923-d62bc8b73e8b'...,archived,2022-09-16T03:01:30.000000Z,2024-07-30T18:25:28.000000Z
4,7c732bd0-ee34-11ee-8057-d3b462e8d235,,,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,None,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,1970-01-01T00:00:00.000000Z,...,,,33,[33],3171 responded to 911 call and provided medica...,1008,[{'id': '679869bd-3b12-485e-ad4e-788df23cab8a'...,archived,2022-09-16T03:51:00.000000Z,2024-07-30T18:25:28.000000Z


## Data Ingest

Now, we can make the http request iteratively to get all the data. We know the total number of records and the number of records per page. We can calculate the number of pages and make the request for each page.

In [3]:
# Make request to get all pages
total_pages = (data_obj['total'] + data_obj['perPage'] - 1) // data_obj['perPage']

# create an empty array to store all data objects
data_objects = []

# loop through all pages
print(f"Total pages: {total_pages}")
for page in range(1, total_pages + 1):
    conn.request("GET", f"/department-api/v/Fire-Billing?page={page}", headers=headers)
    res = conn.getresponse()
    data = res.read()
    data_obj = json.loads(data.decode("utf-8"))
    #print(data_obj['data']) # uncomment to see the data objects
    # take data_obj['data'] array and append it's content (expand array first to get objects) to the data_objects array
    data_objects.extend(data_obj['data'])
    print(f"Page {page} done")

# print the length of the data_objects array
print(f"Data objects length: {len(data_objects)}")

# validate if the length of the data_objects array is equal to the total records
if len(data_objects) == data_obj['total']:
    print("Data objects length is equal to the total records")

# transform data_objects to pandas dataframe
fire_billing_df = pd.DataFrame(data_objects)
fire_billing_df.head(10)
fire_billing_df.tail(10)

Total pages: 883
Page 1 done
Page 2 done
Page 3 done
Page 4 done
Page 5 done
Page 6 done
Page 7 done
Page 8 done
Page 9 done
Page 10 done
Page 11 done
Page 12 done
Page 13 done
Page 14 done
Page 15 done
Page 16 done
Page 17 done
Page 18 done
Page 19 done
Page 20 done
Page 21 done
Page 22 done
Page 23 done
Page 24 done
Page 25 done
Page 26 done
Page 27 done
Page 28 done
Page 29 done
Page 30 done
Page 31 done
Page 32 done
Page 33 done
Page 34 done
Page 35 done
Page 36 done
Page 37 done
Page 38 done
Page 39 done
Page 40 done
Page 41 done
Page 42 done
Page 43 done
Page 44 done
Page 45 done
Page 46 done
Page 47 done
Page 48 done
Page 49 done
Page 50 done
Page 51 done
Page 52 done
Page 53 done
Page 54 done
Page 55 done
Page 56 done
Page 57 done
Page 58 done
Page 59 done
Page 60 done
Page 61 done
Page 62 done
Page 63 done
Page 64 done
Page 65 done
Page 66 done
Page 67 done
Page 68 done
Page 69 done
Page 70 done
Page 71 done
Page 72 done
Page 73 done
Page 74 done
Page 75 done
Page 76 done
Page

,id,Alarm Time,Last Unit Cleared,Incident Number,Incident Type,Address,City,State,Zip,Latitude,...,P2 - Suffix,P2 - City,P2 - Zip Code,Extrication Vehicles,Vehicles,People_Involved,Apparatus,current_status,created_at,updated_at
22043,c29772d0-ee18-11ee-b8cc-d1be69133e5a,,2022-12-22T07:41:22.000000Z,None,321,3720 Laverne,None,None,None,42.191763,...,None,None,None,[],[],[],[{'id': 'c2732b52-1a5f-424d-871c-81b3a88fcb7b'}],archived,2022-12-22T07:33:47.000000Z,2024-03-29T22:07:40.000000Z
22044,c29e25a0-ee18-11ee-a4b5-152aac5751dc,,2022-12-22T15:58:11.000000Z,None,321,126 3RD,None,None,None,42.2219874,...,None,None,None,[],[],[],[{'id': '5a38b118-bd09-4e73-acfc-c47a06838af4'}],archived,2022-12-22T15:44:53.000000Z,2024-03-29T22:07:40.000000Z
22045,c2a3f090-ee18-11ee-9872-f7a7c778d666,,2022-12-22T18:16:45.000000Z,None,321,100 MAIN,None,None,None,42.2213415,...,None,None,None,[],[],[{'id': '1f0117b2-f660-4d30-bb70-4097d00058fc'...,[{'id': 'a7cb067e-5ab5-4174-a416-95de76fe403f'...,archived,2022-12-22T17:58:57.000000Z,2024-03-29T22:07:40.000000Z
22046,c2a9c9e0-ee18-11ee-a4ef-b574ddb33595,,2022-12-22T19:10:54.000000Z,None,321,1325 WIARD,None,None,None,42.216993,...,None,None,None,[],[],[{'id': '1aac0180-ed89-458d-aa2c-8ad22255e710'...,[{'id': '36e672cd-df41-48ea-a732-223de57f5168'...,archived,2022-12-22T18:55:23.000000Z,2024-03-29T22:07:40.000000Z
22047,c2af46b0-ee18-11ee-bbd8-55edad37267d,,2022-12-22T21:37:24.000000Z,None,322,MAIN,None,None,None,42.2283219,...,None,None,None,[],[],[{'id': '78da3e88-1080-4503-a820-a8f4dce2ce5f'...,[{'id': 'a2104bfa-fbce-4b2a-94a0-6ff17d72d579'...,archived,2022-12-22T21:24:25.000000Z,2024-03-29T22:07:40.000000Z
22048,c2b5a520-ee18-11ee-95ae-af16056ff97d,,2022-12-22T23:30:55.000000Z,None,321,2655 SHASTA,None,None,None,42.2163951,...,None,None,None,[],[],[],[{'id': '50393389-0187-458f-b30e-66267dafa4fa'...,archived,2022-12-22T23:18:56.000000Z,2024-03-29T22:07:40.000000Z
22049,c2bb2af0-ee18-11ee-a495-7b33363e06cf,,2022-12-23T01:22:09.000000Z,None,553,323 COMMERCIAL,None,None,None,42.2262219,...,None,None,None,[],[],[],[{'id': '00e67718-5884-48e7-9193-5b65198de3b6'...,archived,2022-12-23T01:14:27.000000Z,2024-03-29T22:07:40.000000Z
22050,c2c10150-ee18-11ee-ab89-c71bbd81f514,,2022-12-23T02:51:02.000000Z,None,611,211 8TH,None,None,None,42.2273809,...,None,None,None,[],[],[{'id': 'ec1dce01-270a-4644-99cf-800576f7d7c5'...,[{'id': 'd7e76871-74e3-4562-905a-c503b1eda48b'...,archived,2022-12-23T02:49:28.000000Z,2024-03-29T22:07:40.000000Z
22051,c2c63ca0-ee18-11ee-97b9-491245277e2d,,2022-12-23T03:57:59.000000Z,None,553,4424 WINTER,None,None,None,42.224867,...,None,None,None,[],[],[{'id': '8c8f3ec3-d192-49b8-abae-116e6bc326f8'...,[{'id': '837b27c0-b40f-4596-9991-463d470ef153'}],archived,2022-12-23T03:37:28.000000Z,2024-03-29T22:07:40.000000Z
22052,c2cbae20-ee18-11ee-bdd8-9d7b9ea9b4f0,,2022-12-23T11:16:38.000000Z,None,321,2455 RADCLIFFE,None,None,None,42.2161349,...,None,None,None,[],[],[{'id': '8a935e21-a7f6-4085-b453-cc29043be85d'...,[{'id': 'bfeb5b1e-e6c2-47fd-a2d7-a189cff7c3d5'...,archived,2022-12-23T10:50:54.000000Z,2024-03-29T22:07:40.000000Z


## Data stats and charts

Here we will show some basic statistics and charts based on the data we have fetched.

In [19]:
# Basic stats and charts

# describe the data
#fire_billing_df.describe()

# aggregate the data by 'incident_type' and count the number of records
fire_billing_df.groupby(['Incident Type', 'Address', 'Incident Narrative']).agg({'id': 'count'}).rename(columns={'id': 'Total Incidents'}).sort_values('Total Incidents', ascending=False).reset_index()


,Incident Type,Address,Incident Narrative,Total Incidents
0,321,2130 ELDORADO,EMS call,37
1,321,615 WASHBURN,EMS call,32
2,321,2865 DAGGETT,Airlink transport,25
3,321,2865 DAGGETT,Airlink transport,23
4,321,615 WASHBURN,EMS call and transport.,23
...,...,...,...,...
19735,321,3000 AIRPORT,Airport transfer,1
19736,321,3000 AIRPORT,Airport Transport- Lifeflight crew transported...,1
19737,321,3000 AIRPORT,Airport Transport with Mercy Flight,1
19738,321,3000 AIRPORT,Airport Transfer,1


## Data modeling and exporting

In [5]:
# Table transformations and key propagation

In [15]:
#export the fire_billing_df to .csv file
fire_billing_df.to_csv('assets/fire_billing.csv', index=False)

#export the fire_billing_df to .json file
fire_billing_df.to_json('assets/fire_billing.json', orient='records')